# YOLOv3

## 상수 정의

In [1]:
TRAIN_SAVE_BEST_ONLY        = True              # val loss가 가장 좋은 모델을 저장, True 권장
TRAIN_SAVE_CHECKPOINT       = False             # True이면 학습 시 모든 유효한 모델을 저장함, False 권장
TRAIN_CLASSES               = "mnist.names"     # 클레스 레이블이 저장되어 있는 파일
TRAIN_ANNOT_PATH            = "mnist_train.txt" # 훈련데이터의 어노테이션 파일 경로
TRAIN_LOGDIR                = "logs"            # 로그 디렉토리
TRAIN_CHECKPOINTS_FOLDER    = "checkpoints"     # 모델이 저장될 디렉토리
TRAIN_MODEL_NAME            = "mnist_custom"    # 저장될 모델의 이름
TRAIN_LOAD_IMAGES_TO_RAM    = True # 더빨리 학습되지만 RAM이 더 필요함
TRAIN_BATCH_SIZE            = 4    # 훈련데이터 배치 크기
TRAIN_INPUT_SIZE            = 416  # 훈련데이터 입력 이미지 크기
TRAIN_DATA_AUG              = True # 훈련 데이터 증강 사용
TRAIN_LR_INIT               = 1e-4 # 훈련시 학습률 초깃값
TRAIN_LR_END                = 1e-6 # 훈련시 학습률 최종값
TRAIN_WARMUP_EPOCHS         = 2    # 워밍업 에포크
TRAIN_EPOCHS                = 100  # 훈련 에포크

TEST_ANNOT_PATH             = "mnist_test.txt"
TEST_BATCH_SIZE             = 4
TEST_INPUT_SIZE             = 416
TEST_DATA_AUG               = False
TEST_SCORE_THRESHOLD        = 0.3
TEST_IOU_THRESHOLD          = 0.45

YOLO_COCO_CLASSES           = "model_data/coco/coco.names"
YOLO_IOU_LOSS_THRESH        = 0.5
YOLO_ANCHOR_PER_SCALE       = 3
YOLO_MAX_BBOX_PER_SCALE     = 100
YOLO_INPUT_SIZE             = 416

# YOLOv3의 합성곱과 Residual

## 라이브러리 불러오기

In [2]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2

## 합성곱함수

### BatchNormalization()

In [3]:
class BatchNormalization(layers.BatchNormalization):
    # "동결 상태(Frozen state)"와 "추론 모드(Inference mode)"는 별개의 개념입니다. 
    # 'layer.trainable=False' 이면 레이어를 동결시킵니다. 이것은 훈련하는 동안 내부 상태 즉, 가중치가 바뀌지 않습니다.
    # 그런데 layer.trainable=False이면 추론 모드로 실행됩니다. 
    # 레이어는 추론모드에서 현재 배치의 평균 및 분산을 사용하는 대신 현재 배치를 정규화하기 위해 이동 평균과 이동 분산을 사용합니다.
    def call(self, x, training=False):
        if not training:
            training = tf.constant(False)
        training = tf.logical_and(training, self.trainable)
        return super().call(x, training)


### convolutional()

In [4]:
def convolutional(input_layer, filters, kernel_size,
                  downsample=False, activate=True, bn=True):
    if downsample:
        input_layer = layers.ZeroPadding2D(((1, 0), (1, 0)))(input_layer)
        padding = 'valid'
        strides = 2
    else:
        strides = 1
        padding = 'same'

    kernel_init = tf.random_normal_initializer(stddev=0.01)
    conv = layers.Conv2D(filters=filters, 
                         kernel_size=kernel_size,
                         strides=strides, padding=padding, 
                         use_bias=not bn,
                         kernel_initializer=kernel_init,
                         kernel_regularizer=l2(0.0005)
                        )(input_layer)
    if bn:
        conv = BatchNormalization()(conv)
    if activate:
        conv = layers.LeakyReLU(alpha=0.1)(conv)

    return conv

## 레지듀얼 블록

### residual_block()

In [5]:
def residual_block(input_layer, filter_num1, filter_num2):
    short_cut = input_layer
    conv = convolutional(input_layer, filters=filter_num1, kernel_size=(1,1))
    conv = convolutional(conv       , filters=filter_num2, kernel_size=(3,3))
    residual_output = short_cut + conv
    return residual_output

## 다크넷 정의

In [6]:
def darknet53(input_data):
    input_data = convolutional(input_data, 32, (3,3))
    input_data = convolutional(input_data, 64, (3,3), downsample=True)

    for i in range(1):
        input_data = residual_block(input_data,  32, 64)

    input_data = convolutional(input_data, 128, (3,3), downsample=True)

    for i in range(2):
        input_data = residual_block(input_data, 64, 128)

    input_data = convolutional(input_data, 256, (3,3), downsample=True)

    for i in range(8):
        input_data = residual_block(input_data, 128, 256)

    route_1 = input_data
    input_data = convolutional(input_data, 512, (3,3), downsample=True)

    for i in range(8):
        input_data = residual_block(input_data, 256, 512)

    route_2 = input_data
    input_data = convolutional(input_data, 1024, (3,3), downsample=True)

    for i in range(4):
        input_data = residual_block(input_data, 512, 1024)

    return route_1, route_2, input_data

## upsample() - 업샘플링

In [7]:
def upsample(input_layer):
    width, height = input_layer.shape[1], input_layer.shape[2]
    output_layer = tf.image.resize(input_layer, (width*2, height*2), 
                                   method='nearest')
    return output_layer

## YOLOv3 함수 정의

In [8]:
def YOLOv3(input_layer, num_class):
    # Darknet-53을 실행하고 그 결과를 받음
    route_1, route_2, conv = darknet53(input_layer)
    
    conv = convolutional(conv, 512, (1,1))
    conv = convolutional(conv, 1024, (3,3))
    conv = convolutional(conv, 512, (1,1))
    conv = convolutional(conv, 1024, (3,3))
    conv = convolutional(conv, 512, (1,1))
    conv_lobj_branch = convolutional(conv, 1024, (3,3))
    
    # conv_lbbox는 큰 객체를 예측하기 위해 사용, Shape = [None, 13, 13, 255] 
    conv_lbbox = convolutional(conv_lobj_branch, 
                               3*(num_class+5), (1,1),
                               activate=False, bn=False)
    
    conv = convolutional(conv, 256, (1,1))
    # 최근방법(nearest)을 이용하여 업샘플링
    # 이렇게 하면 업샘플링시 학습이 필요 없으므로 인공신경망 파라미터를 줄인다.
    conv = upsample(conv)

    conv = tf.concat([conv, route_2], axis=-1)
    conv = convolutional(conv, 256, (1,1))
    conv = convolutional(conv, 512, (3,3))
    conv = convolutional(conv, 256, (1,1))
    conv = convolutional(conv, 512, (3,3))
    conv = convolutional(conv, 256, (1,1))
    conv_mobj_branch = convolutional(conv, 512, (3,3))

    # conv_mbbox는 중간 크기 객체를 예측하기 위해 사용, shape = [None, 26, 26, 255]
    conv_mbbox = convolutional(conv_mobj_branch, 
                               3*(num_class+5), (1,1),
                               activate=False, bn=False)

    conv = convolutional(conv, 128, (1,1))
    conv = upsample(conv)

    conv = tf.concat([conv, route_1], axis=-1)
    conv = convolutional(conv, 128, (1,1))
    conv = convolutional(conv, 256, (3,3))
    conv = convolutional(conv, 128, (1,1))
    conv = convolutional(conv, 256, (3,3))
    conv = convolutional(conv, 128, (1,1))
    conv_sobj_branch = convolutional(conv, 256, (3,3))
    
    # conv_sbbox는 작은 객체를 예측하기 위해 사용, shape = [None, 52, 52, 255]
    conv_sbbox = convolutional(conv_sobj_branch, 
                               3*(num_class+5), (1,1),
                               activate=False, bn=False)
        
    return [conv_sbbox, conv_mbbox, conv_lbbox]

## 합성곱 신경망의 출력을 디코딩 함

In [9]:
import numpy as np

YOLO_STRIDES  = [8, 16, 32]
YOLO_ANCHORS  = [[[10,  13], [16,   30], [33,   23]],
                 [[30,  61], [62,   45], [59,  119]],
                 [[116, 90], [156, 198], [373, 326]]]

STRIDES       = np.array(YOLO_STRIDES)
ANCHORS       = (np.array(YOLO_ANCHORS).T/STRIDES).T

In [10]:
def decode(conv_output, num_class, i=0):
    conv_shape       = tf.shape(conv_output)
    batch_size       = conv_shape[0]
    output_size      = conv_shape[1]

    conv_output = tf.reshape(conv_output, 
                             (batch_size, output_size, output_size, 
                              3, num_class+5))

    conv_raw_dxdy = conv_output[:, :, :, :, 0:2] # 상자의 x, y위치
    conv_raw_dwdh = conv_output[:, :, :, :, 2:4] # 상자의 가로, 세로 크기
    conv_raw_conf = conv_output[:, :, :, :, 4:5] # 상자의 신뢰도(confidence)
    conv_raw_prob = conv_output[:, :, :, :, 5: ] # 클래스별 확률

    # next need Draw the grid. Where output_size is equal to 13, 26 or 52  
    y = tf.range(output_size, dtype=tf.int32)
    y = tf.expand_dims(y, -1)
    y = tf.tile(y, [1, output_size])
    x = tf.range(output_size, dtype=tf.int32)
    x = tf.expand_dims(x, 0)
    x = tf.tile(x, [output_size, 1])

    xy_grid = tf.concat([x[:, :, tf.newaxis], y[:, :, tf.newaxis]], axis=-1)
    xy_grid = tf.tile(xy_grid[tf.newaxis, :, :, tf.newaxis, :], 
                      [batch_size, 1, 1, 3, 1])
    xy_grid = tf.cast(xy_grid, tf.float32)

    # 상자의 중심점을 계산
    pred_xy = (tf.sigmoid(conv_raw_dxdy) + xy_grid) * STRIDES[i]
    # 상자의 너비와 높이를 계산
    pred_wh = (tf.exp(conv_raw_dwdh) * ANCHORS[i]) * STRIDES[i]

    pred_xywh = tf.concat([pred_xy, pred_wh], axis=-1)
    pred_conf = tf.sigmoid(conv_raw_conf) # 상자의 신뢰도 계산
    pred_prob = tf.sigmoid(conv_raw_prob) # 클래스별 확률 계산

    return tf.concat([pred_xywh, pred_conf, pred_prob], axis=-1)

## YOLOv3 모델 생성

In [11]:
NUM_CLASS       = 10

In [12]:
def Create_YOLOv3(input_shape=(416,416,3), train_mode=False, num_class=NUM_CLASS):
    input_layer  = layers.Input(input_shape)
    conv_tensors = YOLOv3(input_layer, num_class)
    output_tensors = []
    for i, conv_tensor in enumerate(conv_tensors):
        pred_tensor = decode(conv_tensor, num_class, i)
        if train_mode: output_tensors.append(conv_tensor)
        output_tensors.append(pred_tensor)

    model = tf.keras.Model(input_layer, output_tensors)
    return model

In [13]:
yolo = Create_YOLOv3(train_mode=True, num_class=NUM_CLASS)
# yolo.summary()

In [14]:
# yolo.load_weights("checkpoints/mnist_custom")

## GPU 사용 설정과 로그 설정

In [15]:
import os
import shutil

gpus = tf.config.experimental.list_physical_devices('GPU')
print(f'GPUs {gpus}')
if len(gpus) > 0:
    try: tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError: pass

if os.path.exists(TRAIN_LOGDIR): shutil.rmtree(TRAIN_LOGDIR)
writer = tf.summary.create_file_writer(TRAIN_LOGDIR)
validate_writer = tf.summary.create_file_writer(TRAIN_LOGDIR)

GPUs [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# 데이터 불러오는 클래스 정의

## Dataset

In [16]:
def read_class_names(class_file_name):
    "파일에서 클래스 레이블 읽어들여 딕셔너리로 만들어 반환함"
    names = {}
    with open(class_file_name, 'r') as data:
        for ID, name in enumerate(data):
            names[ID] = name.strip('\n')
    return names

In [17]:
import random

class Dataset(object):
    # Dataset preprocess implementation
    def __init__(self, dataset_type, TEST_INPUT_SIZE=TEST_INPUT_SIZE):
        self.annot_path  = TRAIN_ANNOT_PATH if dataset_type == 'train' else TEST_ANNOT_PATH
        self.input_sizes = TRAIN_INPUT_SIZE if dataset_type == 'train' else TEST_INPUT_SIZE
        self.batch_size  = TRAIN_BATCH_SIZE if dataset_type == 'train' else TEST_BATCH_SIZE
        self.data_aug    = TRAIN_DATA_AUG   if dataset_type == 'train' else TEST_DATA_AUG

        self.train_input_sizes = TRAIN_INPUT_SIZE
        self.strides = np.array(YOLO_STRIDES)
        self.classes = read_class_names(TRAIN_CLASSES)
        self.num_classes = len(self.classes)
        self.anchors = (np.array(YOLO_ANCHORS).T/self.strides).T
        self.anchor_per_scale = YOLO_ANCHOR_PER_SCALE
        self.max_bbox_per_scale = YOLO_MAX_BBOX_PER_SCALE

        self.annotations = self.load_annotations(dataset_type)
        self.num_samples = len(self.annotations)
        self.num_batchs = int(np.ceil(self.num_samples / self.batch_size))
        self.batch_count = 0


    def load_annotations(self, dataset_type):
        final_annotations = []

        with open(self.annot_path, 'r') as f:
            txt = f.read().splitlines()
            annotations = [line.strip() for line in txt if len(line.strip().split()[1:]) != 0]
        np.random.shuffle(annotations)

        for annotation in annotations:
            line = annotation.split()
            image_path, index = "", 1
            for i, one_line in enumerate(line):
                if not one_line.replace(",","").isnumeric():
                    if image_path != "": image_path += " "
                    image_path += one_line
                else:
                    index = i
                    break
            if not os.path.exists(image_path):
                raise KeyError("%s does not exist ... " %image_path)
            if TRAIN_LOAD_IMAGES_TO_RAM:
                image = cv2.imread(image_path)
            else:
                image = ''
            final_annotations.append([image_path, line[index:], image])
        return final_annotations

    def __iter__(self):
        return self

    def delete_bad_annotation(self, bad_annotation):
        print(f'Deleting {bad_annotation} annotation line')
        bad_image_path = bad_annotation[0]
        bad_image_name = bad_annotation[0].split('/')[-1] # can be used to delete bad image
        bad_xml_path = bad_annotation[0][:-3]+'xml' # can be used to delete bad xml file

        # 잘못된 어노테이션 라인을 삭제함
        with open(self.annot_path, "r+") as f:
            d = f.readlines()
            f.seek(0)
            for i in d:
                if bad_image_name not in i:
                    f.write(i)
            f.truncate()

    def __next__(self):
        with tf.device('/cpu:0'):
            self.train_input_size = random.choice([self.train_input_sizes])
            self.train_output_sizes = self.train_input_size // self.strides

            batch_image = np.zeros((self.batch_size, self.train_input_size, self.train_input_size, 3), dtype=np.float32)

            batch_label_sbbox = np.zeros((self.batch_size, self.train_output_sizes[0], self.train_output_sizes[0], self.anchor_per_scale, 5 + self.num_classes), dtype=np.float32)
            batch_label_mbbox = np.zeros((self.batch_size, self.train_output_sizes[1], self.train_output_sizes[1], self.anchor_per_scale, 5 + self.num_classes), dtype=np.float32)
            batch_label_lbbox = np.zeros((self.batch_size, self.train_output_sizes[2], self.train_output_sizes[2], self.anchor_per_scale, 5 + self.num_classes), dtype=np.float32)

            batch_sbboxes = np.zeros((self.batch_size, self.max_bbox_per_scale, 4), dtype=np.float32)

            batch_mbboxes = np.zeros((self.batch_size, self.max_bbox_per_scale, 4), dtype=np.float32)
            batch_lbboxes = np.zeros((self.batch_size, self.max_bbox_per_scale, 4), dtype=np.float32)

            exceptions = False
            num = 0
            if self.batch_count < self.num_batchs:
                while num < self.batch_size:
                    index = self.batch_count * self.batch_size + num
                    if index >= self.num_samples: index -= self.num_samples
                    annotation = self.annotations[index]
                    image, bboxes = self.parse_annotation(annotation)
                    try:
                        label_sbbox, label_mbbox, label_lbbox, sbboxes, mbboxes, lbboxes = self.preprocess_true_boxes(bboxes)
                    except IndexError:
                        exceptions = True
                        self.delete_bad_annotation(annotation)
                        print("IndexError, something wrong with", annotation[0], "removed this line from annotation file")

                    batch_image[num, :, :, :] = image
                    batch_label_mbbox[num, :, :, :, :] = label_mbbox
                    batch_label_lbbox[num, :, :, :, :] = label_lbbox
                    batch_mbboxes[num, :, :] = mbboxes
                    batch_lbboxes[num, :, :] = lbboxes
                    batch_label_sbbox[num, :, :, :, :] = label_sbbox
                    batch_sbboxes[num, :, :] = sbboxes

                    num += 1

                if exceptions:
                    print('\n')
                    raise Exception("There were problems with dataset, I fixed them, now restart the training process.")
                self.batch_count += 1
                batch_smaller_target = batch_label_sbbox, batch_sbboxes
                batch_medium_target  = batch_label_mbbox, batch_mbboxes
                batch_larger_target  = batch_label_lbbox, batch_lbboxes

                return batch_image, (batch_smaller_target, batch_medium_target, batch_larger_target)
            else:
                self.batch_count = 0
                np.random.shuffle(self.annotations)
                raise StopIteration

    def random_horizontal_flip(self, image, bboxes):
        if random.random() < 0.5:
            _, w, _ = image.shape
            image = image[:, ::-1, :]
            bboxes[:, [0,2]] = w - bboxes[:, [2,0]]

        return image, bboxes

    def random_crop(self, image, bboxes):
        if random.random() < 0.5:
            h, w, _ = image.shape
            max_bbox = np.concatenate([np.min(bboxes[:, 0:2], axis=0), np.max(bboxes[:, 2:4], axis=0)], axis=-1)

            max_l_trans = max_bbox[0]
            max_u_trans = max_bbox[1]
            max_r_trans = w - max_bbox[2]
            max_d_trans = h - max_bbox[3]

            crop_xmin = max(0, int(max_bbox[0] - random.uniform(0, max_l_trans)))
            crop_ymin = max(0, int(max_bbox[1] - random.uniform(0, max_u_trans)))
            crop_xmax = max(w, int(max_bbox[2] + random.uniform(0, max_r_trans)))
            crop_ymax = max(h, int(max_bbox[3] + random.uniform(0, max_d_trans)))

            image = image[crop_ymin : crop_ymax, crop_xmin : crop_xmax]

            bboxes[:, [0, 2]] = bboxes[:, [0, 2]] - crop_xmin
            bboxes[:, [1, 3]] = bboxes[:, [1, 3]] - crop_ymin

        return image, bboxes

    def random_translate(self, image, bboxes):
        if random.random() < 0.5:
            h, w, _ = image.shape
            max_bbox = np.concatenate([np.min(bboxes[:, 0:2], axis=0), np.max(bboxes[:, 2:4], axis=0)], axis=-1)

            max_l_trans = max_bbox[0]
            max_u_trans = max_bbox[1]
            max_r_trans = w - max_bbox[2]
            max_d_trans = h - max_bbox[3]

            tx = random.uniform(-(max_l_trans - 1), (max_r_trans - 1))
            ty = random.uniform(-(max_u_trans - 1), (max_d_trans - 1))

            M = np.array([[1, 0, tx], [0, 1, ty]])
            image = cv2.warpAffine(image, M, (w, h))

            bboxes[:, [0, 2]] = bboxes[:, [0, 2]] + tx
            bboxes[:, [1, 3]] = bboxes[:, [1, 3]] + ty

        return image, bboxes

    def parse_annotation(self, annotation, mAP = 'False'):
        if TRAIN_LOAD_IMAGES_TO_RAM:
            image_path = annotation[0]
            image = annotation[2]
        else:
            image_path = annotation[0]
            image = cv2.imread(image_path)

        bboxes = np.array([list(map(int, box.split(','))) for box in annotation[1]])

        if self.data_aug:
            image, bboxes = self.random_horizontal_flip(np.copy(image), np.copy(bboxes))
            image, bboxes = self.random_crop(np.copy(image), np.copy(bboxes))
            image, bboxes = self.random_translate(np.copy(image), np.copy(bboxes))

        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if mAP == True:
            return image, bboxes

        image, bboxes = image_preprocess(np.copy(image), [self.input_sizes, self.input_sizes], np.copy(bboxes))
        return image, bboxes

    def preprocess_true_boxes(self, bboxes):
        OUTPUT_LEVELS = len(self.strides)

        label = [np.zeros((self.train_output_sizes[i], self.train_output_sizes[i], self.anchor_per_scale,
                           5 + self.num_classes)) for i in range(OUTPUT_LEVELS)]
        bboxes_xywh = [np.zeros((self.max_bbox_per_scale, 4)) for _ in range(OUTPUT_LEVELS)]
        bbox_count = np.zeros((OUTPUT_LEVELS,))

        for bbox in bboxes:
            bbox_coor = bbox[:4]
            bbox_class_ind = bbox[4]

            onehot = np.zeros(self.num_classes, dtype=np.float64)
            onehot[bbox_class_ind] = 1.0
            uniform_distribution = np.full(self.num_classes, 1.0 / self.num_classes)
            deta = 0.01
            smooth_onehot = onehot * (1 - deta) + deta * uniform_distribution

            bbox_xywh = np.concatenate([(bbox_coor[2:] + bbox_coor[:2]) * 0.5, bbox_coor[2:] - bbox_coor[:2]], axis=-1)
            bbox_xywh_scaled = 1.0 * bbox_xywh[np.newaxis, :] / self.strides[:, np.newaxis]

            iou = []
            exist_positive = False
            for i in range(OUTPUT_LEVELS):#range(3):
                anchors_xywh = np.zeros((self.anchor_per_scale, 4))
                anchors_xywh[:, 0:2] = np.floor(bbox_xywh_scaled[i, 0:2]).astype(np.int32) + 0.5
                anchors_xywh[:, 2:4] = self.anchors[i]

                iou_scale = bbox_iou(bbox_xywh_scaled[i][np.newaxis, :], anchors_xywh)
                iou.append(iou_scale)
                iou_mask = iou_scale > 0.3

                if np.any(iou_mask):
                    xind, yind = np.floor(bbox_xywh_scaled[i, 0:2]).astype(np.int32)

                    label[i][yind, xind, iou_mask, :] = 0
                    label[i][yind, xind, iou_mask, 0:4] = bbox_xywh
                    label[i][yind, xind, iou_mask, 4:5] = 1.0
                    label[i][yind, xind, iou_mask, 5:] = smooth_onehot

                    bbox_ind = int(bbox_count[i] % self.max_bbox_per_scale)
                    bboxes_xywh[i][bbox_ind, :4] = bbox_xywh
                    bbox_count[i] += 1

                    exist_positive = True

            if not exist_positive:
                best_anchor_ind = np.argmax(np.array(iou).reshape(-1), axis=-1)
                best_detect = int(best_anchor_ind / self.anchor_per_scale)
                best_anchor = int(best_anchor_ind % self.anchor_per_scale)
                xind, yind = np.floor(bbox_xywh_scaled[best_detect, 0:2]).astype(np.int32)

                label[best_detect][yind, xind, best_anchor, :] = 0
                label[best_detect][yind, xind, best_anchor, 0:4] = bbox_xywh
                label[best_detect][yind, xind, best_anchor, 4:5] = 1.0
                label[best_detect][yind, xind, best_anchor, 5:] = smooth_onehot

                bbox_ind = int(bbox_count[best_detect] % self.max_bbox_per_scale)
                bboxes_xywh[best_detect][bbox_ind, :4] = bbox_xywh
                bbox_count[best_detect] += 1

        label_sbbox, label_mbbox, label_lbbox = label
        sbboxes, mbboxes, lbboxes = bboxes_xywh
        return label_sbbox, label_mbbox, label_lbbox, sbboxes, mbboxes, lbboxes

    def __len__(self):
        return self.num_batchs


## train_step & validate_step

In [18]:
def train_step(image_data, target):
    with tf.GradientTape() as tape:
        pred_result = yolo(image_data, training=True)
        giou_loss=conf_loss=prob_loss=0

        # optimizing process
        grid = 3
        for i in range(grid):
            conv, pred = pred_result[i*2], pred_result[i*2+1]
            loss_items = compute_loss(pred, conv, *target[i], i, CLASSES=TRAIN_CLASSES)
            giou_loss += loss_items[0]
            conf_loss += loss_items[1]
            prob_loss += loss_items[2]

        total_loss = giou_loss + conf_loss + prob_loss

        gradients = tape.gradient(total_loss, yolo.trainable_variables)
        optimizer.apply_gradients(zip(gradients, yolo.trainable_variables))

        # update learning rate
        # about warmup: https://arxiv.org/pdf/1812.01187.pdf&usg=ALkJrhglKOPDjNt6SHGbphTHyMcT0cuMJg
        global_steps.assign_add(1)
        if global_steps < warmup_steps:
            lr = global_steps / warmup_steps * TRAIN_LR_INIT
        else:
            lr = TRAIN_LR_END + 0.5 * (TRAIN_LR_INIT - TRAIN_LR_END)*(
                (1 + tf.cos((global_steps - warmup_steps) / (total_steps - warmup_steps) * np.pi)))
        optimizer.lr.assign(lr.numpy())

        # writing summary data
        with writer.as_default():
            tf.summary.scalar("lr", optimizer.lr, step=global_steps)
            tf.summary.scalar("loss/total_loss", total_loss, step=global_steps)
            tf.summary.scalar("loss/giou_loss", giou_loss, step=global_steps)
            tf.summary.scalar("loss/conf_loss", conf_loss, step=global_steps)
            tf.summary.scalar("loss/prob_loss", prob_loss, step=global_steps)
        writer.flush()
        
    return global_steps.numpy(), optimizer.lr.numpy(), giou_loss.numpy(), conf_loss.numpy(), prob_loss.numpy(), total_loss.numpy()

In [19]:
def validate_step(image_data, target):
    with tf.GradientTape() as tape:
        pred_result = yolo(image_data, training=False)
        giou_loss=conf_loss=prob_loss=0

        # optimizing process
        grid = 3 
        for i in range(grid):
            conv, pred = pred_result[i*2], pred_result[i*2+1]
            loss_items = compute_loss(pred, conv, *target[i], i, CLASSES=TRAIN_CLASSES)
            giou_loss += loss_items[0]
            conf_loss += loss_items[1]
            prob_loss += loss_items[2]

        total_loss = giou_loss + conf_loss + prob_loss
        
    return giou_loss.numpy(), conf_loss.numpy(), prob_loss.numpy(), total_loss.numpy()

## image_preprocess

In [20]:
import cv2

def image_preprocess(image, target_size, gt_boxes=None):
    ih, iw    = target_size
    h,  w, _  = image.shape

    scale = min(iw/w, ih/h)
    nw, nh  = int(scale * w), int(scale * h)
    image_resized = cv2.resize(image, (nw, nh))

    image_paded = np.full(shape=[ih, iw, 3], fill_value=128.0)
    dw, dh = (iw - nw) // 2, (ih-nh) // 2
    image_paded[dh:nh+dh, dw:nw+dw, :] = image_resized
    image_paded = image_paded / 255.

    if gt_boxes is None:
        return image_paded

    else:
        gt_boxes[:, [0, 2]] = gt_boxes[:, [0, 2]] * scale + dw
        gt_boxes[:, [1, 3]] = gt_boxes[:, [1, 3]] * scale + dh
        return image_paded, gt_boxes


## bbox_iou

In [21]:
def bbox_iou(boxes1, boxes2):
    boxes1_area = boxes1[..., 2] * boxes1[..., 3]
    boxes2_area = boxes2[..., 2] * boxes2[..., 3]

    boxes1 = tf.concat([boxes1[..., :2] - boxes1[..., 2:] * 0.5,
                        boxes1[..., :2] + boxes1[..., 2:] * 0.5], axis=-1)
    boxes2 = tf.concat([boxes2[..., :2] - boxes2[..., 2:] * 0.5,
                        boxes2[..., :2] + boxes2[..., 2:] * 0.5], axis=-1)

    left_up = tf.maximum(boxes1[..., :2], boxes2[..., :2])
    right_down = tf.minimum(boxes1[..., 2:], boxes2[..., 2:])

    inter_section = tf.maximum(right_down - left_up, 0.0)
    inter_area = inter_section[..., 0] * inter_section[..., 1]
    union_area = boxes1_area + boxes2_area - inter_area

    return 1.0 * inter_area / union_area

def bbox_giou(boxes1, boxes2):
    boxes1 = tf.concat([boxes1[..., :2] - boxes1[..., 2:] * 0.5,
                        boxes1[..., :2] + boxes1[..., 2:] * 0.5], axis=-1)
    boxes2 = tf.concat([boxes2[..., :2] - boxes2[..., 2:] * 0.5,
                        boxes2[..., :2] + boxes2[..., 2:] * 0.5], axis=-1)

    boxes1 = tf.concat([tf.minimum(boxes1[..., :2], boxes1[..., 2:]),
                        tf.maximum(boxes1[..., :2], boxes1[..., 2:])], axis=-1)
    boxes2 = tf.concat([tf.minimum(boxes2[..., :2], boxes2[..., 2:]),
                        tf.maximum(boxes2[..., :2], boxes2[..., 2:])], axis=-1)

    boxes1_area = (boxes1[..., 2] - boxes1[..., 0]) * (boxes1[..., 3] - boxes1[..., 1])
    boxes2_area = (boxes2[..., 2] - boxes2[..., 0]) * (boxes2[..., 3] - boxes2[..., 1])

    left_up = tf.maximum(boxes1[..., :2], boxes2[..., :2])
    right_down = tf.minimum(boxes1[..., 2:], boxes2[..., 2:])

    inter_section = tf.maximum(right_down - left_up, 0.0)
    inter_area = inter_section[..., 0] * inter_section[..., 1]
    union_area = boxes1_area + boxes2_area - inter_area

    # Calculate the iou value between the two bounding boxes
    iou = inter_area / union_area

    # Calculate the coordinates of the upper left corner and the lower right corner of the smallest closed convex surface
    enclose_left_up = tf.minimum(boxes1[..., :2], boxes2[..., :2])
    enclose_right_down = tf.maximum(boxes1[..., 2:], boxes2[..., 2:])
    enclose = tf.maximum(enclose_right_down - enclose_left_up, 0.0)

    # Calculate the area of the smallest closed convex surface C
    enclose_area = enclose[..., 0] * enclose[..., 1]

    # Calculate the GIoU value according to the GioU formula  
    giou = iou - 1.0 * (enclose_area - union_area) / enclose_area

    return giou

# testing (should be better than giou)
def bbox_ciou(boxes1, boxes2):
    boxes1_coor = tf.concat([boxes1[..., :2] - boxes1[..., 2:] * 0.5,
                        boxes1[..., :2] + boxes1[..., 2:] * 0.5], axis=-1)
    boxes2_coor = tf.concat([boxes2[..., :2] - boxes2[..., 2:] * 0.5,
                        boxes2[..., :2] + boxes2[..., 2:] * 0.5], axis=-1)

    left = tf.maximum(boxes1_coor[..., 0], boxes2_coor[..., 0])
    up = tf.maximum(boxes1_coor[..., 1], boxes2_coor[..., 1])
    right = tf.maximum(boxes1_coor[..., 2], boxes2_coor[..., 2])
    down = tf.maximum(boxes1_coor[..., 3], boxes2_coor[..., 3])

    c = (right - left) * (right - left) + (up - down) * (up - down)
    iou = bbox_iou(boxes1, boxes2)

    u = (boxes1[..., 0] - boxes2[..., 0]) * (boxes1[..., 0] - boxes2[..., 0]) + (boxes1[..., 1] - boxes2[..., 1]) * (boxes1[..., 1] - boxes2[..., 1])
    d = u / c

    ar_gt = boxes2[..., 2] / boxes2[..., 3]
    ar_pred = boxes1[..., 2] / boxes1[..., 3]

    ar_loss = 4 / (np.pi * np.pi) * (tf.atan(ar_gt) - tf.atan(ar_pred)) * (tf.atan(ar_gt) - tf.atan(ar_pred))
    alpha = ar_loss / (1 - iou + ar_loss + 0.000001)
    ciou_term = d + alpha * ar_loss

    return iou - ciou_term


## compute_loss

In [22]:
def compute_loss(pred, conv, label, bboxes, i=0, CLASSES=YOLO_COCO_CLASSES):
    NUM_CLASS = len(read_class_names(CLASSES))
    conv_shape  = tf.shape(conv)
    batch_size  = conv_shape[0]
    output_size = conv_shape[1]
    input_size  = STRIDES[i] * output_size
    conv = tf.reshape(conv, (batch_size, output_size, output_size, 3, 5 + NUM_CLASS))

    conv_raw_conf = conv[:, :, :, :, 4:5]
    conv_raw_prob = conv[:, :, :, :, 5:]

    pred_xywh     = pred[:, :, :, :, 0:4]
    pred_conf     = pred[:, :, :, :, 4:5]

    label_xywh    = label[:, :, :, :, 0:4]
    respond_bbox  = label[:, :, :, :, 4:5]
    label_prob    = label[:, :, :, :, 5:]

    giou = tf.expand_dims(bbox_giou(pred_xywh, label_xywh), axis=-1)
    input_size = tf.cast(input_size, tf.float32)

    bbox_loss_scale = 2.0 - 1.0 * label_xywh[:, :, :, :, 2:3] * label_xywh[:, :, :, :, 3:4] / (input_size ** 2)
    giou_loss = respond_bbox * bbox_loss_scale * (1 - giou)

    iou = bbox_iou(pred_xywh[:, :, :, :, np.newaxis, :], bboxes[:, np.newaxis, np.newaxis, np.newaxis, :, :])
    # Find the value of IoU with the real box The largest prediction box
    max_iou = tf.expand_dims(tf.reduce_max(iou, axis=-1), axis=-1)

    # If the largest iou is less than the threshold, it is considered that the prediction box contains no objects, then the background box
    respond_bgd = (1.0 - respond_bbox) * tf.cast( max_iou < YOLO_IOU_LOSS_THRESH, tf.float32 )

    conf_focal = tf.pow(respond_bbox - pred_conf, 2)

    # Calculate the loss of confidence
    # we hope that if the grid contains objects, then the network output prediction box has a confidence of 1 and 0 when there is no object.
    conf_loss = conf_focal * (
            respond_bbox * tf.nn.sigmoid_cross_entropy_with_logits(labels=respond_bbox, logits=conv_raw_conf)
            +
            respond_bgd * tf.nn.sigmoid_cross_entropy_with_logits(labels=respond_bbox, logits=conv_raw_conf)
    )

    prob_loss = respond_bbox * tf.nn.sigmoid_cross_entropy_with_logits(labels=label_prob, logits=conv_raw_prob)

    giou_loss = tf.reduce_mean(tf.reduce_sum(giou_loss, axis=[1,2,3,4]))
    conf_loss = tf.reduce_mean(tf.reduce_sum(conf_loss, axis=[1,2,3,4]))
    prob_loss = tf.reduce_mean(tf.reduce_sum(prob_loss, axis=[1,2,3,4]))

    return giou_loss, conf_loss, prob_loss


## 데이터 불러와 학습시키기

In [23]:
trainset = Dataset('train')
testset = Dataset('test')
steps_per_epoch = len(trainset)
global_steps = tf.Variable(1, trainable=False, dtype=tf.int64)
warmup_steps = TRAIN_WARMUP_EPOCHS * steps_per_epoch
total_steps = TRAIN_EPOCHS * steps_per_epoch

optimizer = tf.keras.optimizers.Adam()

In [24]:
best_val_loss = 1000 # should be large at start
for epoch in range(TRAIN_EPOCHS):
    for image_data, target in trainset:
        results = train_step(image_data, target)
        cur_step = results[0]%steps_per_epoch
        print("epoch:{:2.0f} step:{:5.0f}/{}, lr:{:.6f}, giou_loss:{:7.2f}, conf_loss:{:7.2f}, prob_loss:{:7.2f}, total_loss:{:7.2f}"
              .format(epoch, cur_step, steps_per_epoch, results[1], results[2], results[3], results[4], results[5]))

    if len(testset) == 0:
        print("configure TEST options to validate model")
        yolo.save_weights(os.path.join(TRAIN_CHECKPOINTS_FOLDER, TRAIN_MODEL_NAME))
        continue
    
    count, giou_val, conf_val, prob_val, total_val = 0., 0, 0, 0, 0
    for image_data, target in testset:
        results = validate_step(image_data, target)
        count += 1
        giou_val += results[0]
        conf_val += results[1]
        prob_val += results[2]
        total_val += results[3]
    # writing validate summary data
    with validate_writer.as_default():
        tf.summary.scalar("validate_loss/total_val", total_val/count, step=epoch)
        tf.summary.scalar("validate_loss/giou_val", giou_val/count, step=epoch)
        tf.summary.scalar("validate_loss/conf_val", conf_val/count, step=epoch)
        tf.summary.scalar("validate_loss/prob_val", prob_val/count, step=epoch)
    validate_writer.flush()
        
    print("\n\ngiou_val_loss:{:7.2f}, conf_val_loss:{:7.2f}, prob_val_loss:{:7.2f}, total_val_loss:{:7.2f}\n\n".
          format(giou_val/count, conf_val/count, prob_val/count, total_val/count))

    if TRAIN_SAVE_CHECKPOINT and not TRAIN_SAVE_BEST_ONLY:
        save_directory = os.path.join(TRAIN_CHECKPOINTS_FOLDER, TRAIN_MODEL_NAME+"_val_loss_{:7.2f}".format(total_val/count))
        yolo.save_weights(save_directory)
    if TRAIN_SAVE_BEST_ONLY and best_val_loss>total_val/count:
        save_directory = os.path.join(TRAIN_CHECKPOINTS_FOLDER, TRAIN_MODEL_NAME)
        yolo.save_weights(save_directory)
        best_val_loss = total_val/count
    if not TRAIN_SAVE_BEST_ONLY and not TRAIN_SAVE_CHECKPOINT:
        save_directory = os.path.join(TRAIN_CHECKPOINTS_FOLDER, TRAIN_MODEL_NAME)
        yolo.save_weights(save_directory)



epoch: 0 step:    2/250, lr:0.000000, giou_loss:  17.29, conf_loss:1788.69, prob_loss: 106.14, total_loss:1912.13
epoch: 0 step:    3/250, lr:0.000001, giou_loss:  23.16, conf_loss:1761.15, prob_loss: 103.76, total_loss:1888.07
epoch: 0 step:    4/250, lr:0.000001, giou_loss:  18.45, conf_loss:1724.86, prob_loss: 115.68, total_loss:1858.99
epoch: 0 step:    5/250, lr:0.000001, giou_loss:  28.55, conf_loss:1736.98, prob_loss: 137.77, total_loss:1903.31
epoch: 0 step:    6/250, lr:0.000001, giou_loss:  14.39, conf_loss:1713.15, prob_loss:  90.90, total_loss:1818.44
epoch: 0 step:    7/250, lr:0.000001, giou_loss:  23.51, conf_loss:1759.75, prob_loss: 134.75, total_loss:1918.01
epoch: 0 step:    8/250, lr:0.000002, giou_loss:  16.52, conf_loss:1712.14, prob_loss:  75.43, total_loss:1804.10
epoch: 0 step:    9/250, lr:0.000002, giou_loss:  19.66, conf_loss:1695.08, prob_loss:  90.45, total_loss:1805.18
epoch: 0 step:   10/250, lr:0.000002, giou_loss:  25.28, conf_loss:1731.13, prob_loss: 1

epoch: 0 step:   74/250, lr:0.000015, giou_loss:   9.91, conf_loss:1119.97, prob_loss:  58.07, total_loss:1187.95
epoch: 0 step:   75/250, lr:0.000015, giou_loss:  15.73, conf_loss:1127.04, prob_loss:  89.99, total_loss:1232.77
epoch: 0 step:   76/250, lr:0.000015, giou_loss:  15.97, conf_loss:1115.56, prob_loss:  91.33, total_loss:1222.86
epoch: 0 step:   77/250, lr:0.000015, giou_loss:  14.01, conf_loss:1104.93, prob_loss:  75.55, total_loss:1194.48
epoch: 0 step:   78/250, lr:0.000016, giou_loss:  14.41, conf_loss:1102.97, prob_loss:  69.91, total_loss:1187.30
epoch: 0 step:   79/250, lr:0.000016, giou_loss:  11.32, conf_loss:1092.59, prob_loss:  65.36, total_loss:1169.27
epoch: 0 step:   80/250, lr:0.000016, giou_loss:  19.52, conf_loss:1096.14, prob_loss: 114.99, total_loss:1230.65
epoch: 0 step:   81/250, lr:0.000016, giou_loss:  13.35, conf_loss:1080.87, prob_loss:  68.91, total_loss:1163.14
epoch: 0 step:   82/250, lr:0.000016, giou_loss:   9.64, conf_loss:1063.76, prob_loss:  

epoch: 0 step:  146/250, lr:0.000029, giou_loss:  15.94, conf_loss: 762.26, prob_loss:  89.32, total_loss: 867.52
epoch: 0 step:  147/250, lr:0.000029, giou_loss:  12.47, conf_loss: 755.15, prob_loss:  67.42, total_loss: 835.04
epoch: 0 step:  148/250, lr:0.000030, giou_loss:  11.91, conf_loss: 742.60, prob_loss:  56.72, total_loss: 811.23
epoch: 0 step:  149/250, lr:0.000030, giou_loss:  13.36, conf_loss: 734.25, prob_loss:  74.75, total_loss: 822.36
epoch: 0 step:  150/250, lr:0.000030, giou_loss:  10.15, conf_loss: 731.38, prob_loss:  54.49, total_loss: 796.02
epoch: 0 step:  151/250, lr:0.000030, giou_loss:  11.93, conf_loss: 724.29, prob_loss:  66.15, total_loss: 802.38
epoch: 0 step:  152/250, lr:0.000030, giou_loss:  14.50, conf_loss: 734.73, prob_loss:  75.31, total_loss: 824.54
epoch: 0 step:  153/250, lr:0.000031, giou_loss:  11.44, conf_loss: 715.44, prob_loss:  63.90, total_loss: 790.77
epoch: 0 step:  154/250, lr:0.000031, giou_loss:   9.81, conf_loss: 719.12, prob_loss:  

epoch: 0 step:  218/250, lr:0.000044, giou_loss:   9.15, conf_loss: 471.74, prob_loss:  56.52, total_loss: 537.40
epoch: 0 step:  219/250, lr:0.000044, giou_loss:  11.59, conf_loss: 465.36, prob_loss:  59.00, total_loss: 535.95
epoch: 0 step:  220/250, lr:0.000044, giou_loss:  15.27, conf_loss: 475.54, prob_loss:  79.32, total_loss: 570.13
epoch: 0 step:  221/250, lr:0.000044, giou_loss:  11.94, conf_loss: 473.62, prob_loss:  71.37, total_loss: 556.94
epoch: 0 step:  222/250, lr:0.000044, giou_loss:  11.12, conf_loss: 460.16, prob_loss:  63.41, total_loss: 534.69
epoch: 0 step:  223/250, lr:0.000045, giou_loss:   7.63, conf_loss: 450.04, prob_loss:  41.68, total_loss: 499.35
epoch: 0 step:  224/250, lr:0.000045, giou_loss:   7.88, conf_loss: 446.85, prob_loss:  40.88, total_loss: 495.60
epoch: 0 step:  225/250, lr:0.000045, giou_loss:  13.82, conf_loss: 452.75, prob_loss:  79.21, total_loss: 545.78
epoch: 0 step:  226/250, lr:0.000045, giou_loss:   9.46, conf_loss: 447.30, prob_loss:  

epoch: 1 step:   40/250, lr:0.000058, giou_loss:  11.28, conf_loss: 286.66, prob_loss:  55.40, total_loss: 353.35
epoch: 1 step:   41/250, lr:0.000058, giou_loss:  12.33, conf_loss: 287.56, prob_loss:  65.16, total_loss: 365.06
epoch: 1 step:   42/250, lr:0.000058, giou_loss:   9.54, conf_loss: 284.49, prob_loss:  51.73, total_loss: 345.76
epoch: 1 step:   43/250, lr:0.000059, giou_loss:   8.30, conf_loss: 279.72, prob_loss:  44.46, total_loss: 332.48
epoch: 1 step:   44/250, lr:0.000059, giou_loss:  10.34, conf_loss: 283.11, prob_loss:  54.98, total_loss: 348.43
epoch: 1 step:   45/250, lr:0.000059, giou_loss:  10.04, conf_loss: 277.13, prob_loss:  57.48, total_loss: 344.65
epoch: 1 step:   46/250, lr:0.000059, giou_loss:  11.04, conf_loss: 277.71, prob_loss:  69.75, total_loss: 358.50
epoch: 1 step:   47/250, lr:0.000059, giou_loss:   9.20, conf_loss: 274.14, prob_loss:  51.99, total_loss: 335.33
epoch: 1 step:   48/250, lr:0.000060, giou_loss:   5.74, conf_loss: 268.68, prob_loss:  

...

epoch:20 step:  186/250, lr:0.000091, giou_loss:   3.78, conf_loss:   1.11, prob_loss:   5.31, total_loss:  10.20
epoch:20 step:  187/250, lr:0.000091, giou_loss:   3.61, conf_loss:   0.55, prob_loss:   4.06, total_loss:   8.23
epoch:20 step:  188/250, lr:0.000091, giou_loss:   4.97, conf_loss:   0.86, prob_loss:  12.69, total_loss:  18.53
epoch:20 step:  189/250, lr:0.000091, giou_loss:   6.61, conf_loss:   0.80, prob_loss:   7.94, total_loss:  15.35
epoch:20 step:  190/250, lr:0.000091, giou_loss:   5.33, conf_loss:   1.46, prob_loss:   7.53, total_loss:  14.31
epoch:20 step:  191/250, lr:0.000091, giou_loss:   3.47, conf_loss:   0.83, prob_loss:   6.08, total_loss:  10.38
epoch:20 step:  192/250, lr:0.000091, giou_loss:   5.70, conf_loss:   1.02, prob_loss:  10.51, total_loss:  17.24
epoch:20 step:  193/250, lr:0.000091, giou_loss:   5.90, conf_loss:   1.69, prob_loss:  14.03, total_loss:  21.62
epoch:20 step:  194/250, lr:0.000091, giou_loss:   6.61, conf_loss:   1.21, prob_loss:  

epoch:21 step:    8/250, lr:0.000091, giou_loss:   7.00, conf_loss:   1.63, prob_loss:   9.56, total_loss:  18.19
epoch:21 step:    9/250, lr:0.000091, giou_loss:   4.24, conf_loss:   0.97, prob_loss:   9.47, total_loss:  14.68
epoch:21 step:   10/250, lr:0.000091, giou_loss:   4.94, conf_loss:   0.54, prob_loss:   6.94, total_loss:  12.42
epoch:21 step:   11/250, lr:0.000091, giou_loss:   5.80, conf_loss:   0.73, prob_loss:   4.12, total_loss:  10.65
epoch:21 step:   12/250, lr:0.000091, giou_loss:   4.44, conf_loss:   0.85, prob_loss:   5.02, total_loss:  10.31
epoch:21 step:   13/250, lr:0.000091, giou_loss:   5.12, conf_loss:   1.01, prob_loss:   9.34, total_loss:  15.47
epoch:21 step:   14/250, lr:0.000091, giou_loss:   5.71, conf_loss:   1.01, prob_loss:   6.16, total_loss:  12.88
epoch:21 step:   15/250, lr:0.000091, giou_loss:   4.47, conf_loss:   0.80, prob_loss:   5.37, total_loss:  10.64
epoch:21 step:   16/250, lr:0.000091, giou_loss:   4.53, conf_loss:   0.54, prob_loss:  

epoch:21 step:   80/250, lr:0.000091, giou_loss:   5.07, conf_loss:   0.99, prob_loss:   8.52, total_loss:  14.57
epoch:21 step:   81/250, lr:0.000091, giou_loss:   4.19, conf_loss:   3.63, prob_loss:  16.05, total_loss:  23.87
epoch:21 step:   82/250, lr:0.000091, giou_loss:   3.74, conf_loss:   0.92, prob_loss:   5.43, total_loss:  10.09
epoch:21 step:   83/250, lr:0.000091, giou_loss:   4.76, conf_loss:   0.75, prob_loss:  12.75, total_loss:  18.26
epoch:21 step:   84/250, lr:0.000091, giou_loss:   3.50, conf_loss:   0.43, prob_loss:   6.44, total_loss:  10.37
epoch:21 step:   85/250, lr:0.000091, giou_loss:   5.99, conf_loss:   2.97, prob_loss:   7.73, total_loss:  16.69
epoch:21 step:   86/250, lr:0.000091, giou_loss:   3.68, conf_loss:   0.91, prob_loss:   2.80, total_loss:   7.39
epoch:21 step:   87/250, lr:0.000091, giou_loss:   6.25, conf_loss:   1.10, prob_loss:   9.95, total_loss:  17.30
epoch:21 step:   88/250, lr:0.000091, giou_loss:   6.16, conf_loss:   1.79, prob_loss:  

epoch:21 step:  152/250, lr:0.000091, giou_loss:   5.11, conf_loss:   2.45, prob_loss:   8.05, total_loss:  15.60
epoch:21 step:  153/250, lr:0.000091, giou_loss:   4.42, conf_loss:   1.02, prob_loss:   7.91, total_loss:  13.35
epoch:21 step:  154/250, lr:0.000091, giou_loss:   5.71, conf_loss:   1.15, prob_loss:   6.05, total_loss:  12.90
epoch:21 step:  155/250, lr:0.000091, giou_loss:   5.29, conf_loss:   0.71, prob_loss:   4.82, total_loss:  10.82
epoch:21 step:  156/250, lr:0.000091, giou_loss:   4.56, conf_loss:   0.81, prob_loss:   4.50, total_loss:   9.88
epoch:21 step:  157/250, lr:0.000091, giou_loss:   5.20, conf_loss:   0.81, prob_loss:   5.04, total_loss:  11.04


KeyboardInterrupt: 

In [25]:
# def get_mAP(Yolo, dataset, score_threshold=0.25, iou_threshold=0.50, TEST_INPUT_SIZE=TEST_INPUT_SIZE):
#     MINOVERLAP = 0.5 # default value (defined in the PASCAL VOC2012 challenge)
#     NUM_CLASS = read_class_names(TRAIN_CLASSES)

#     ground_truth_dir_path = 'mAP/ground-truth'
#     if os.path.exists(ground_truth_dir_path): shutil.rmtree(ground_truth_dir_path)

#     if not os.path.exists('mAP'): os.mkdir('mAP')
#     os.mkdir(ground_truth_dir_path)

#     print(f'\ncalculating mAP{int(iou_threshold*100)}...\n')

#     gt_counter_per_class = {}
#     for index in range(dataset.num_samples):
#         ann_dataset = dataset.annotations[index]

#         original_image, bbox_data_gt = dataset.parse_annotation(ann_dataset, True)

#         if len(bbox_data_gt) == 0:
#             bboxes_gt = []
#             classes_gt = []
#         else:
#             bboxes_gt, classes_gt = bbox_data_gt[:, :4], bbox_data_gt[:, 4]
#         ground_truth_path = os.path.join(ground_truth_dir_path, str(index) + '.txt')
#         num_bbox_gt = len(bboxes_gt)

#         bounding_boxes = []
#         for i in range(num_bbox_gt):
#             class_name = NUM_CLASS[classes_gt[i]]
#             xmin, ymin, xmax, ymax = list(map(str, bboxes_gt[i]))
#             bbox = xmin + " " + ymin + " " + xmax + " " +ymax
#             bounding_boxes.append({"class_name":class_name, "bbox":bbox, "used":False})

#             # count that object
#             if class_name in gt_counter_per_class:
#                 gt_counter_per_class[class_name] += 1
#             else:
#                 # if class didn't exist yet
#                 gt_counter_per_class[class_name] = 1
#             bbox_mess = ' '.join([class_name, xmin, ymin, xmax, ymax]) + '\n'
#         with open(f'{ground_truth_dir_path}/{str(index)}_ground_truth.json', 'w') as outfile:
#             json.dump(bounding_boxes, outfile)

#     gt_classes = list(gt_counter_per_class.keys())
#     # sort the classes alphabetically
#     gt_classes = sorted(gt_classes)
#     n_classes = len(gt_classes)

#     times = []
#     json_pred = [[] for i in range(n_classes)]
#     for index in range(dataset.num_samples):
#         ann_dataset = dataset.annotations[index]

#         image_name = ann_dataset[0].split('/')[-1]
#         original_image, bbox_data_gt = dataset.parse_annotation(ann_dataset, True)
        
#         image = image_preprocess(np.copy(original_image), [TEST_INPUT_SIZE, TEST_INPUT_SIZE])
#         image_data = image[np.newaxis, ...].astype(np.float32)

#         t1 = time.time()
#         if YOLO_FRAMEWORK == "tf":
#             pred_bbox = Yolo.predict(image_data)
#         elif YOLO_FRAMEWORK == "trt":
#             batched_input = tf.constant(image_data)
#             result = Yolo(batched_input)
#             pred_bbox = []
#             for key, value in result.items():
#                 value = value.numpy()
#                 pred_bbox.append(value)
        
#         t2 = time.time()
        
#         times.append(t2-t1)
        
#         pred_bbox = [tf.reshape(x, (-1, tf.shape(x)[-1])) for x in pred_bbox]
#         pred_bbox = tf.concat(pred_bbox, axis=0)

#         bboxes = postprocess_boxes(pred_bbox, original_image, TEST_INPUT_SIZE, score_threshold)
#         bboxes = nms(bboxes, iou_threshold, method='nms')

#         for bbox in bboxes:
#             coor = np.array(bbox[:4], dtype=np.int32)
#             score = bbox[4]
#             class_ind = int(bbox[5])
#             class_name = NUM_CLASS[class_ind]
#             score = '%.4f' % score
#             xmin, ymin, xmax, ymax = list(map(str, coor))
#             bbox = xmin + " " + ymin + " " + xmax + " " +ymax
#             json_pred[gt_classes.index(class_name)].append({"confidence": str(score), "file_id": str(index), "bbox": str(bbox)})

#     ms = sum(times)/len(times)*1000
#     fps = 1000 / ms

#     for class_name in gt_classes:
#         json_pred[gt_classes.index(class_name)].sort(key=lambda x:float(x['confidence']), reverse=True)
#         with open(f'{ground_truth_dir_path}/{class_name}_predictions.json', 'w') as outfile:
#             json.dump(json_pred[gt_classes.index(class_name)], outfile)

#     # Calculate the AP for each class
#     sum_AP = 0.0
#     ap_dictionary = {}
#     # open file to store the results
#     with open("mAP/results.txt", 'w') as results_file:
#         results_file.write("# AP and precision/recall per class\n")
#         count_true_positives = {}
#         for class_index, class_name in enumerate(gt_classes):
#             count_true_positives[class_name] = 0
#             # Load predictions of that class
#             predictions_file = f'{ground_truth_dir_path}/{class_name}_predictions.json'
#             predictions_data = json.load(open(predictions_file))

#             # Assign predictions to ground truth objects
#             nd = len(predictions_data)
#             tp = [0] * nd # creates an array of zeros of size nd
#             fp = [0] * nd
#             for idx, prediction in enumerate(predictions_data):
#                 file_id = prediction["file_id"]
#                 # assign prediction to ground truth object if any
#                 #   open ground-truth with that file_id
#                 gt_file = f'{ground_truth_dir_path}/{str(file_id)}_ground_truth.json'
#                 ground_truth_data = json.load(open(gt_file))
#                 ovmax = -1
#                 gt_match = -1
#                 # load prediction bounding-box
#                 bb = [ float(x) for x in prediction["bbox"].split() ] # bounding box of prediction
#                 for obj in ground_truth_data:
#                     # look for a class_name match
#                     if obj["class_name"] == class_name:
#                         bbgt = [ float(x) for x in obj["bbox"].split() ] # bounding box of ground truth
#                         bi = [max(bb[0],bbgt[0]), max(bb[1],bbgt[1]), min(bb[2],bbgt[2]), min(bb[3],bbgt[3])]
#                         iw = bi[2] - bi[0] + 1
#                         ih = bi[3] - bi[1] + 1
#                         if iw > 0 and ih > 0:
#                             # compute overlap (IoU) = area of intersection / area of union
#                             ua = (bb[2] - bb[0] + 1) * (bb[3] - bb[1] + 1) + (bbgt[2] - bbgt[0]
#                                             + 1) * (bbgt[3] - bbgt[1] + 1) - iw * ih
#                             ov = iw * ih / ua
#                             if ov > ovmax:
#                                 ovmax = ov
#                                 gt_match = obj

#                 # assign prediction as true positive/don't care/false positive
#                 if ovmax >= MINOVERLAP:# if ovmax > minimum overlap
#                     if not bool(gt_match["used"]):
#                         # true positive
#                         tp[idx] = 1
#                         gt_match["used"] = True
#                         count_true_positives[class_name] += 1
#                         # update the ".json" file
#                         with open(gt_file, 'w') as f:
#                             f.write(json.dumps(ground_truth_data))
#                     else:
#                         # false positive (multiple detection)
#                         fp[idx] = 1
#                 else:
#                     # false positive
#                     fp[idx] = 1

#             # compute precision/recall
#             cumsum = 0
#             for idx, val in enumerate(fp):
#                 fp[idx] += cumsum
#                 cumsum += val
#             cumsum = 0
#             for idx, val in enumerate(tp):
#                 tp[idx] += cumsum
#                 cumsum += val
#             #print(tp)
#             rec = tp[:]
#             for idx, val in enumerate(tp):
#                 rec[idx] = float(tp[idx]) / gt_counter_per_class[class_name]
#             #print(rec)
#             prec = tp[:]
#             for idx, val in enumerate(tp):
#                 prec[idx] = float(tp[idx]) / (fp[idx] + tp[idx])
#             #print(prec)

#             ap, mrec, mprec = voc_ap(rec, prec)
#             sum_AP += ap
#             text = "{0:.3f}%".format(ap*100) + " = " + class_name + " AP  " #class_name + " AP = {0:.2f}%".format(ap*100)

#             rounded_prec = [ '%.3f' % elem for elem in prec ]
#             rounded_rec = [ '%.3f' % elem for elem in rec ]
#             # Write to results.txt
#             results_file.write(text + "\n Precision: " + str(rounded_prec) + "\n Recall   :" + str(rounded_rec) + "\n\n")

#             print(text)
#             ap_dictionary[class_name] = ap

#         results_file.write("\n# mAP of all classes\n")
#         mAP = sum_AP / n_classes

#         text = "mAP = {:.3f}%, {:.2f} FPS".format(mAP*100, fps)
#         results_file.write(text + "\n")
#         print(text)
        
#         return mAP*100

In [26]:
# # measure mAP of trained custom model
# mAP_model = Create_YOLOv3()
# try:
#     mAP_model.load_weights(save_directory) # use keras weights
#     get_mAP(mAP_model, testset, score_threshold=TEST_SCORE_THRESHOLD, iou_threshold=TEST_IOU_THRESHOLD)
# except UnboundLocalError:
#     print("You don't have saved model weights to measure mAP, check TRAIN_SAVE_BEST_ONLY and TRAIN_SAVE_CHECKPOINT lines in configs.py")
    

In [27]:
# yolo.save_weights("checkpoints/mnist_custom")

# 학습 후

## 박스 후처리(postprocess_boxes)

In [28]:
def read_class_names(class_file_name):
    "파일에서 클래스 레이블 읽어들여 딕셔너리로 만들어 반환함"
    names = {}
    with open(class_file_name, 'r') as data:
        for ID, name in enumerate(data):
            names[ID] = name.strip('\n')
    return names

In [29]:
def image_preprocess(image, target_size, gt_boxes=None):
    ih, iw    = target_size
    h,  w, _  = image.shape

    scale = min(iw/w, ih/h)
    nw, nh  = int(scale * w), int(scale * h)
    image_resized = cv2.resize(image, (nw, nh))

    image_paded = np.full(shape=[ih, iw, 3], fill_value=128.0)
    dw, dh = (iw - nw) // 2, (ih-nh) // 2
    image_paded[dh:nh+dh, dw:nw+dw, :] = image_resized
    image_paded = image_paded / 255.

    if gt_boxes is None:
        return image_paded

    else:
        gt_boxes[:, [0, 2]] = gt_boxes[:, [0, 2]] * scale + dw
        gt_boxes[:, [1, 3]] = gt_boxes[:, [1, 3]] * scale + dh
        return image_paded, gt_boxes


In [30]:
def postprocess_boxes(pred_bbox, original_image, input_size, score_threshold):
    valid_scale=[0, np.inf]
    pred_bbox = np.array(pred_bbox)

    pred_xywh = pred_bbox[:, 0:4]
    pred_conf = pred_bbox[:, 4]
    pred_prob = pred_bbox[:, 5:]

    # 1. (x, y, w, h) --> (xmin, ymin, xmax, ymax)
    pred_coor = np.concatenate([pred_xywh[:, :2] - pred_xywh[:, 2:] * 0.5,
                                pred_xywh[:, :2] + pred_xywh[:, 2:] * 0.5], axis=-1)
    # 2. (xmin, ymin, xmax, ymax) -> (xmin_org, ymin_org, xmax_org, ymax_org)
    org_h, org_w = original_image.shape[:2]
    resize_ratio = min(input_size / org_w, input_size / org_h)

    dw = (input_size - resize_ratio * org_w) / 2
    dh = (input_size - resize_ratio * org_h) / 2

    pred_coor[:, 0::2] = 1.0 * (pred_coor[:, 0::2] - dw) / resize_ratio
    pred_coor[:, 1::2] = 1.0 * (pred_coor[:, 1::2] - dh) / resize_ratio

    # 3. clip some boxes those are out of range
    pred_coor = np.concatenate([np.maximum(pred_coor[:, :2], [0, 0]),
                                np.minimum(pred_coor[:, 2:], [org_w - 1, org_h - 1])], axis=-1)
    invalid_mask = np.logical_or((pred_coor[:, 0] > pred_coor[:, 2]), (pred_coor[:, 1] > pred_coor[:, 3]))
    pred_coor[invalid_mask] = 0

    # 4. discard some invalid boxes
    bboxes_scale = np.sqrt(np.multiply.reduce(pred_coor[:, 2:4] - pred_coor[:, 0:2], axis=-1))
    scale_mask = np.logical_and((valid_scale[0] < bboxes_scale), (bboxes_scale < valid_scale[1]))

    # 5. discard boxes with low scores
    classes = np.argmax(pred_prob, axis=-1)
    scores = pred_conf * pred_prob[np.arange(len(pred_coor)), classes]
    score_mask = scores > score_threshold
    mask = np.logical_and(scale_mask, score_mask)
    coors, scores, classes = pred_coor[mask], scores[mask], classes[mask]

    return np.concatenate([coors, scores[:, np.newaxis], classes[:, np.newaxis]], axis=-1)


## bboxes_iou

In [31]:
def bboxes_iou(boxes1, boxes2):
    boxes1 = np.array(boxes1)
    boxes2 = np.array(boxes2)

    boxes1_area = (boxes1[..., 2] - boxes1[..., 0]) * (boxes1[..., 3] - boxes1[..., 1])
    boxes2_area = (boxes2[..., 2] - boxes2[..., 0]) * (boxes2[..., 3] - boxes2[..., 1])

    left_up       = np.maximum(boxes1[..., :2], boxes2[..., :2])
    right_down    = np.minimum(boxes1[..., 2:], boxes2[..., 2:])

    inter_section = np.maximum(right_down - left_up, 0.0)
    inter_area    = inter_section[..., 0] * inter_section[..., 1]
    union_area    = boxes1_area + boxes2_area - inter_area
    ious          = np.maximum(1.0 * inter_area / union_area, np.finfo(np.float32).eps)

    return ious


## nms

In [32]:
def nms(bboxes, iou_threshold, sigma=0.3, method='nms'):
    """
    param bboxes: 튜플, (xmin, ymin, xmax, ymax, score, class) 형식
    iou_threshold: float 타입, IoU 스레스홀드
    sigma: Soft-NMS에서 사용, 기본값 0.3
    method: 'nms'와 'soft-nms'를 사용할 수 있음

    참고문헌: soft-nms, https://arxiv.org/pdf/1704.04503.pdf
          https://github.com/bharatsingh430/soft-nms
    """
    classes_in_img = list(set(bboxes[:, 5]))
    best_bboxes = []

    for cls in classes_in_img:
        cls_mask = (bboxes[:, 5] == cls)
        cls_bboxes = bboxes[cls_mask]
        # 1. 경계 상자의 개수가 0보다 큰지 확인
        while len(cls_bboxes) > 0:
            # 2. 점수 순서 A에 따라 가장 높은 점수를 갖는 경계 상자를 선택
            max_ind = np.argmax(cls_bboxes[:, 4])
            best_bbox = cls_bboxes[max_ind]
            best_bboxes.append(best_bbox)
            cls_bboxes = np.concatenate([cls_bboxes[: max_ind], cls_bboxes[max_ind + 1:]])
            # 3. 경계 상자 A를 계산하고 경계 상자의 모든 iou를 계산하고 iou 값이 임계값보다 높은 경계 상자를 제거
            iou = bboxes_iou(best_bbox[np.newaxis, :4], cls_bboxes[:, :4])
            weight = np.ones((len(iou),), dtype=np.float32)

            assert method in ['nms', 'soft-nms']

            if method == 'nms':
                iou_mask = iou > iou_threshold
                weight[iou_mask] = 0.0

            if method == 'soft-nms':
                weight = np.exp(-(1.0 * iou ** 2 / sigma))

            cls_bboxes[:, 4] = cls_bboxes[:, 4] * weight
            score_mask = cls_bboxes[:, 4] > 0.
            cls_bboxes = cls_bboxes[score_mask]

    return best_bboxes

## draw_bbox

In [33]:
import colorsys

def draw_bbox(image, bboxes, CLASSES=YOLO_COCO_CLASSES, show_label=True, show_confidence = True, Text_colors=(255,255,0), rectangle_colors='', tracking=False):   
    NUM_CLASS = read_class_names(CLASSES)
    num_classes = len(NUM_CLASS)
    image_h, image_w, _ = image.shape
    hsv_tuples = [(1.0 * x / num_classes, 1., 1.) for x in range(num_classes)]
    #print("hsv_tuples", hsv_tuples)
    colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
    colors = list(map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)), colors))

    random.seed(0)
    random.shuffle(colors)
    random.seed(None)

    for i, bbox in enumerate(bboxes):
        coor = np.array(bbox[:4], dtype=np.int32)
        score = bbox[4]
        class_ind = int(bbox[5])
        bbox_color = rectangle_colors if rectangle_colors != '' else colors[class_ind]
        bbox_thick = int(0.6 * (image_h + image_w) / 1000)
        if bbox_thick < 1: bbox_thick = 1
        fontScale = 0.75 * bbox_thick
        (x1, y1), (x2, y2) = (coor[0], coor[1]), (coor[2], coor[3])

        # put object rectangle
        cv2.rectangle(image, (x1, y1), (x2, y2), bbox_color, bbox_thick*2)

        if show_label:
            # get text label
            score_str = " {:.2f}".format(score) if show_confidence else ""

            if tracking: score_str = " "+str(score)

            try:
                label = "{}".format(NUM_CLASS[class_ind]) + score_str
            except KeyError:
                print("You received KeyError, this might be that you are trying to use yolo original weights")
                print("while using custom classes, if using custom model in configs.py set YOLO_CUSTOM_WEIGHTS = True")

            # get text size
            (text_width, text_height), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_COMPLEX_SMALL,
                                                                  fontScale, thickness=bbox_thick)
            # put filled text rectangle
            cv2.rectangle(image, (x1, y1), (x1 + text_width, y1 - text_height - baseline), bbox_color, thickness=cv2.FILLED)

            # put text above rectangle
            cv2.putText(image, label, (x1, y1-4), cv2.FONT_HERSHEY_COMPLEX_SMALL,
                        fontScale, Text_colors, bbox_thick, lineType=cv2.LINE_AA)

    return image


## detect_image

In [34]:
yolo = Create_YOLOv3()
yolo.load_weights("checkpoints/mnist_custom")
weight = yolo.get_weights()

In [35]:
def detect_image(Yolo, image_path, output_path, input_size=416, show=False, CLASSES=YOLO_COCO_CLASSES, score_threshold=0.3, iou_threshold=0.45, rectangle_colors=''):
    original_image      = cv2.imread(image_path)
#     original_image      = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
#     original_image      = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

    image_data = image_preprocess(np.copy(original_image), [input_size, input_size])
    image_data = image_data[np.newaxis, ...].astype(np.float32)
#     print(image_data.shape)
    pred_bbox = Yolo.predict(image_data)
#     for i in range(len(pred_bbox)):
#         print(pred_bbox[i].shape, end=' ')
#     print()

    pred_bbox = [tf.reshape(x, (-1, tf.shape(x)[-1])) for x in pred_bbox]
#     pred_bbox = [tf.reshape(x, (-1, tf.shape(x)[-1])) for i, x in enumerate(pred_bbox) if i%2==1]
    pred_bbox = tf.concat(pred_bbox, axis=0)
    
    bboxes = postprocess_boxes(pred_bbox, original_image, input_size, score_threshold)
    bboxes = nms(bboxes, iou_threshold, method='nms')

    image = draw_bbox(original_image, bboxes, CLASSES=CLASSES, rectangle_colors=rectangle_colors)
    # CreateXMLfile("XML_Detections", str(int(time.time())), original_image, bboxes, read_class_names(CLASSES))

    if output_path != '': cv2.imwrite(output_path, image)
    if show:
        cv2.imshow("predicted image", image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
    return image

## detect

In [36]:
yolo.set_weights(weight)
result_image = detect_image(yolo, "mnist_test_c.jpg", "mnist_test_out.jpg", 
                            input_size=416, show=True, 
                            CLASSES=TRAIN_CLASSES, rectangle_colors=(255,0,0))